# CESM JSON to SQL + Outlier Detection Thesis 1

10/23/19  
The purpose of this project is to establish a remote connection to a SQL database housed on AWS and populated with usage data from CESM, and to prepare the data for analysis by performing outlier detection and data type confirmation using pandas.

CESM data was collected by parsing timing files with the run.sh script, created by software engineering at NCAR, and saving the results as a JSON file. This notebook extracts the JSON data using pandas, parses an extra column and sets column data types, and writes the table to the SQL server. The resulting data is then cleaned for outliers and written as a .csv called df_NoOutlier.csv.

This notebook also creates a CPU_hours column and run_length_years column.

This notebook uses data from 9/10/19.

**If loading new data, be sure to change the paths and filenames in cells 3, 16, 17, and 24!**
**Don't overwrite your old data.**

## Load Required Packages

In [1]:
#Load packages

import sqlalchemy as sa
import pandas as pd
import json
import matplotlib.pyplot as plt

## Establish Database Connection

In [2]:
#Create engine address  
engine = sa.create_engine('mysql://dbuser:xxx/cesm')

#Create connection
connection = engine.connect()

#Test connection
#Print list of tables in db
print (engine.table_names())

['CESM_Data', 'CESM_Data_09_10_2019', 'CESM_Data_2', 'CESM_Data_3', 'CESM_Data_Thesis', 'CESM_GladeWork_Test', 'GladeDataTest', 'GladeData_Test', 't2_cases', 't2_cmip6_DECK_exps', 't2_cmip6_DECK_types', 't2_cmip6_MIP_types', 't2_cmip6_exps', 't2_cmip6_forcings', 't2_cmip6_init', 't2_cmip6_physics', 't2_cmip6_source_id', 't2_cmip6_sources', 't2_expType', 't2_linkType', 't2_process', 't2_publish_types', 't2_status', 't2e_fields', 't2e_notes', 't2j_cmip6', 't2j_cmip6_exps_mips', 't2j_cmip6_source_types', 't2j_links', 't2j_publish_approvals', 't2j_status']


## Read Entries (JSON)
**If loading new data, it's important here to set the .json input file path and name.  

In [3]:
#****
#Set file path where .json file resides
path = "/Users/admin/Documents/NCAR/cmip_09_10_2019/"

#Open json file and save as dictionary
#****
#Set file name
with open(path+'cmip_09_10_2019.json') as json_file:  
    full_dict = json.load(json_file)
    
#Create df from dictionary
df = pd.DataFrame(list(full_dict.values()), index=full_dict.keys())

#Confirm
df.head()

,LID,Timeroot,case,caseroot,comp_pes_atm,comp_pes_cpl,comp_pes_esp,comp_pes_glc,comp_pes_ice,comp_pes_lnd,...,threads_cpl,threads_esp,threads_glc,threads_ice,threads_lnd,threads_ocn,threads_rof,threads_wav,total_pes_active,user
/glade/scratch/bdobbins/projects/Database/CMIPFiles/cesm_timing.b.e21.BWSSP126cmip6.f09_g17.CMIP6-SSP1-2.6-WACCM.001.4142239.chadmin1.190127-015112,4142239.chadmin1.190127-015112,/glade/work/cmip6/cases/ScenarioMIP_WACCM/b.e2...,b.e21.BWSSP126cmip6.f09_g17.CMIP6-SSP1-2.6-WAC...,/glade/work/cmip6/cases/ScenarioMIP_WACCM/b.e2...,3456,3456,1,3456,756,2592,...,3,1,3,3,3,3,3,3,10692,cmip6
/glade/scratch/bdobbins/projects/Database/CMIPFiles/cesm_timing.b.e21.BW1850.f09_g17.CMIP6-piControl.001.1220262.chadmin1.180723-102337,1220262.chadmin1.180723-102337,/glade/work/cmip6/cases/cesm2.1-exp002/b.e21.B...,b.e21.BW1850.f09_g17.CMIP6-piControl.001,/glade/work/cmip6/cases/cesm2.1-exp002/b.e21.B...,6912,6912,6,6912,1728,4968,...,6,6,6,6,6,6,6,6,42336,cmip6
/glade/scratch/bdobbins/projects/Database/CMIPFiles/cesm_timing.b.e21.BWSSP585cmip6.f09_g17.CMIP6-G6solar-WACCM.002.8170274.chadmin1.ib0.cheyenne.ucar.edu.190907-005735,8170274.chadmin1.ib0.cheyenne.ucar.edu.190907-...,/gpfs/fs1/work/cmip6/cases/GeoMIP/b.e21.BWSSP5...,b.e21.BWSSP585cmip6.f09_g17.CMIP6-G6solar-WACC...,/gpfs/fs1/work/cmip6/cases/GeoMIP/b.e21.BWSSP5...,3456,3456,1,3456,756,2592,...,3,1,3,3,3,3,3,3,10692,cmip6
/glade/scratch/bdobbins/projects/Database/CMIPFiles/cesm_timing.f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-pdSST-futAntSIC.096.4711042.chadmin1.190317-181426,4711042.chadmin1.190317-181426,/gpfs/fs1/work/cmip6/cases/PAMIP/f.e21.FHIST_B...,f.e21.FHIST_BGC.f09_f09_mg17.CMIP6-pdSST-futAn...,/gpfs/fs1/work/cmip6/cases/PAMIP/f.e21.FHIST_B...,1080,1080,1,1080,1080,1080,...,3,1,3,3,3,3,3,3,3240,cmip6
/glade/scratch/bdobbins/projects/Database/CMIPFiles/cesm_timing.b.e21.BWSSP370lowNTCFcmip6.f09_g17.CMIP6-SSP3-7.0-lowNTCF-WACCM.002.7137706.chadmin1.ib0.cheyenne.ucar.edu.190713-212357,7137706.chadmin1.ib0.cheyenne.ucar.edu.190713-...,/glade/work/cmip6/cases/AerChemMIP/b.e21.BWSSP...,b.e21.BWSSP370lowNTCFcmip6.f09_g17.CMIP6-SSP3-...,/glade/work/cmip6/cases/AerChemMIP/b.e21.BWSSP...,3456,3456,1,3456,756,2592,...,3,1,3,3,3,3,3,3,10692,cmip6


In [4]:
#Count number of entries
df.shape

(6088, 97)

In [5]:
#Set the index into its own Id column and create new index 
df.index.names = ['Id']
df = df.reset_index()
pd.set_option('display.max_colwidth', -1)
with pd.option_context("display.max_columns", 1000):
    display(df)

,Id,LID,Timeroot,case,caseroot,comp_pes_atm,comp_pes_cpl,comp_pes_esp,comp_pes_glc,comp_pes_ice,comp_pes_lnd,comp_pes_ocn,comp_pes_rof,comp_pes_wav,component_atm,component_cpl,component_esp,component_glc,component_ice,component_lnd,component_ocn,component_rof,component_wav,compset,curr_date,final_time,grid,init_time_seconds,instances_atm,instances_cpl,instances_esp,instances_glc,instances_ice,instances_lnd,instances_ocn,instances_rof,instances_wav,machine,model_cost,model_throughput,ocn_init_wait_time,parser,pe_count,pes_per_node,root_pe_atm,root_pe_cpl,root_pe_esp,root_pe_glc,root_pe_ice,root_pe_lnd,root_pe_ocn,root_pe_rof,root_pe_wav,run_length,run_time_atm,run_time_cpl,run_time_cplcomm,run_time_esp,run_time_glc,run_time_ice,run_time_lnd,run_time_ocn,run_time_rof,run_time_seconds,run_time_seconds_per_day,run_time_tot,run_time_wav,run_type,stop_option,stride_atm,stride_cpl,stride_esp,stride_glc,stride_ice,stride_lnd,stride_ocn,stride_rof,stride_wav,tasks_atm,tasks_cpl,tasks_esp,tasks_glc,tasks_ice,tasks_lnd,tasks_ocn,tasks_rof,tasks_wav,threads_atm,threads_cpl,threads_esp,threads_glc,threads_ice,threads_lnd,threads_ocn,threads_rof,threads_wav,total_pes_active,user
0,/glade/scratch/bdobbins/projects/Database/CMIPFiles/cesm_timing.b.e21.BWSSP126cmip6.f09_g17.CMIP6-SSP1-2.6-WACCM.001.4142239.chadmin1.190127-015112,4142239.chadmin1.190127-015112,/glade/work/cmip6/cases/ScenarioMIP_WACCM/b.e21.BWSSP126cmip6.f09_g17.CMIP6-SSP1-2.6-WACCM.001/Tools,b.e21.BWSSP126cmip6.f09_g17.CMIP6-SSP1-2.6-WACCM.001,/glade/work/cmip6/cases/ScenarioMIP_WACCM/b.e21.BWSSP126cmip6.f09_g17.CMIP6-SSP1-2.6-WACCM.001,3456,3456,1,3456,756,2592,108,2592,108,cam,cpl,sesp,cism,cice,clm,pop,mosart,ww,SSP126_CAM60%WCTS_CLM50%BGC-CROP-CMIP6WACCMDECK_CICE%CMIP6_POP2%ECO%NDEP_MOSART_CISM2%NOEVOLVE_WW3,Sun Jan 27 07:56:41 2019,0.005,a%0.9x1.25_l%0.9x1.25_oi%gx1v7_r%r05_g%gland4_w%ww3a_m%gx1v7,98.480,1,1,1,1,1,1,1,1,1,cheyenne,21599.85,3.96,9236.843,cesm2b.template,3564,36,0,0,0,0,864,0,1152,0,1116,365 days (365.0 for ocean),20409.275,260.630,1031.757,0.000,1.087,757.028,802.498,12532.518,54.789,21818.035,59.775,21818.035,466.638,"hybrid, continue_run = TRUE (inittype = FALSE)","nyears, stop_n = 1",1,1,1,1,1,1,1,1,1,1152,1152,1,1152,252,864,36,864,36,3,3,1,3,3,3,3,3,3,10692,cmip6
1,/glade/scratch/bdobbins/projects/Database/CMIPFiles/cesm_timing.b.e21.BW1850.f09_g17.CMIP6-piControl.001.1220262.chadmin1.180723-102337,1220262.chadmin1.180723-102337,/glade/work/cmip6/cases/cesm2.1-exp002/b.e21.BW1850.f09_g17.CMIP6-piControl.001/Tools,b.e21.BW1850.f09_g17.CMIP6-piControl.001,/glade/work/cmip6/cases/cesm2.1-exp002/b.e21.BW1850.f09_g17.CMIP6-piControl.001,6912,6912,6,6912,1728,4968,144,4968,216,cam,cpl,sesp,cism,cice,clm,pop,mosart,ww,1850_CAM60%WCTS_CLM50%BGC-CROP_CICE_POP2%ECO%NDEP_MOSART_CISM2%NOEVOLVE_WW3,Mon Jul 23 14:37:47 2018,0.024,a%0.9x1.25_l%0.9x1.25_oi%gx1v7_r%r05_g%gland4_w%ww3a_m%gx1v7,95.478,1,1,1,1,1,1,1,1,1,cheyenne,29654.06,5.71,4790.734,cesm2b.template,7056,36,0,0,0,0,828,0,1152,0,1116,365 days (365.0 for ocean),13982.685,338.130,712.238,0.000,1.158,340.555,676.310,10268.046,28.452,15129.621,41.451,15129.621,465.757,"hybrid, continue_run = TRUE (inittype = FALSE)","nyears, stop_n = 1",1,1,1,1,1,1,1,1,1,1152,1152,1,1152,288,828,24,828,36,6,6,6,6,6,6,6,6,6,42336,cmip6
2,/glade/scratch/bdobbins/projects/Database/CMIPFiles/cesm_timing.b.e21.BWSSP585cmip6.f09_g17.CMIP6-G6solar-WACCM.002.8170274.chadmin1.ib0.cheyenne.ucar.edu.190907-005735,8170274.chadmin1.ib0.cheyenne.ucar.edu.190907-005735,/gpfs/fs1/work/cmip6/cases/GeoMIP/b.e21.BWSSP585cmip6.f09_g17.CMIP6-G6solar-WACCM.002/Tools,b.e21.BWSSP585cmip6.f09_g17.CMIP6-G6solar-WACCM.002,/gpfs/fs1/work/cmip6/cases/GeoMIP/b.e21.BWSSP585cmip6.f09_g17.CMIP6-G6solar-WACCM.002,3456,3456,1,3456,756,2592,108,2592,108,cam,cpl,sesp,cism,cice,clm,pop,mosart,ww,SSP585_CAM60%WCTS_CLM50%BGC-CROP-CMIP6WACCMDECK_CICE%CMIP6_POP2%ECO%NDEP_MOSART_CISM2%NOEVOLVE_WW3,Sat Sep 7 06:40:14 2019,0.051,a%0.9x1.25_l%0.9x1.25_oi%gx1v7_r%r05_g%g

## Parse Run_Length

Parse numeric values from the run_length column (length of climate simulation):

In [6]:
df.run_length.head()

0    365 days (365.0 for ocean)        
1    365 days (365.0 for ocean)        
2    365 days (365.0 for ocean)        
3    426 days (425.979166667 for ocean)
4    365 days (365.0 for ocean)        
Name: run_length, dtype: object

In [7]:
## Parse Run_Length

#1. Strip everything after "days" in run_length column
df['run_length_temp'] = df['run_length'].str.split('(').str[0]

#Confirm every run_length contains the same units of days
substr = 'days'
print("Rows in df:", len(df))
print ("Rows with units of days:", df.run_length_temp.str.count(substr).sum())

Rows in df: 6088
Rows with units of days: 6088


In [8]:
#2. Strip "days" in run_length column

#Name new column run_length_days
df['run_length_days'] = df['run_length_temp'].str.split(' d').str[0]
df.run_length_days.unique()

array(['365', '426', '3650', '1825', '2190', '31', '730', '1095', '5840',
       '7300', '5475', '4014', '90', '5', '2189', '2', '10950', '1460',
       '4379', '1'], dtype=object)

In [9]:
#Delete run_length and run_length_temp
del df['run_length']
del df['run_length_temp']

In [10]:
#Confirm
df.columns

Index(['Id', 'LID', 'Timeroot', 'case', 'caseroot', 'comp_pes_atm',
       'comp_pes_cpl', 'comp_pes_esp', 'comp_pes_glc', 'comp_pes_ice',
       'comp_pes_lnd', 'comp_pes_ocn', 'comp_pes_rof', 'comp_pes_wav',
       'component_atm', 'component_cpl', 'component_esp', 'component_glc',
       'component_ice', 'component_lnd', 'component_ocn', 'component_rof',
       'component_wav', 'compset', 'curr_date', 'final_time', 'grid',
       'init_time_seconds', 'instances_atm', 'instances_cpl', 'instances_esp',
       'instances_glc', 'instances_ice', 'instances_lnd', 'instances_ocn',
       'instances_rof', 'instances_wav', 'machine', 'model_cost',
       'model_throughput', 'ocn_init_wait_time', 'parser', 'pe_count',
       'pes_per_node', 'root_pe_atm', 'root_pe_cpl', 'root_pe_esp',
       'root_pe_glc', 'root_pe_ice', 'root_pe_lnd', 'root_pe_ocn',
       'root_pe_rof', 'root_pe_wav', 'run_time_atm', 'run_time_cpl',
       'run_time_cplcomm', 'run_time_esp', 'run_time_glc', 'run_time_ice',


## Set Data Types

In [11]:
#Look for features that may need converted to numeric or date

with pd.option_context("display.max_rows", 1000):
    print(df.dtypes)

Id                          object
LID                         object
Timeroot                    object
case                        object
caseroot                    object
comp_pes_atm                object
comp_pes_cpl                object
comp_pes_esp                object
comp_pes_glc                object
comp_pes_ice                object
comp_pes_lnd                object
comp_pes_ocn                object
comp_pes_rof                object
comp_pes_wav                object
component_atm               object
component_cpl               object
component_esp               object
component_glc               object
component_ice               object
component_lnd               object
component_ocn               object
component_rof               object
component_wav               object
compset                     object
curr_date                   object
final_time                  object
grid                        object
init_time_seconds           object
instances_atm       

In [12]:
#Automatically convert columns to numeric format
df = df.apply(pd.to_numeric, errors='ignore')

#Convert necessary curr_date to date format
df['curr_date'] = pd.to_datetime(df.curr_date)

In [13]:
#Create run_length_years column: 
df['run_length_years'] = df['run_length_days'] / 365
df.run_length_years.describe()

count    6088.000000
mean     3.578388   
std      4.170578   
min      0.002740   
25%      1.000000   
50%      1.167123   
75%      5.000000   
max      30.000000  
Name: run_length_years, dtype: float64

In [15]:
df.run_length_years.sum()

21785.224657534243

In [ ]:
#CPU hours per run = model_cost_num (in pe hours/simulated year) * simulated years in that run  
df['CPU_hours'] = df['model_cost'] * df['run_length_years']

In [ ]:
#Confirm data types
with pd.option_context("display.max_rows", 1000):
    print(df.dtypes)

## Write to SQL Db
It's also important here to set the new SQL table name.

In [ ]:
#Write df to SQL db
#****
#Set new table name
df.to_sql("CESM_Data_Thesis", if_exists = 'replace', index=False, con=connection)

#Confirm table in SQL
print (engine.table_names())

In [ ]:
#Confirm reading from SQL
#Read and print CESM_Data table
#****
#Set new SQL table name
df_CESM_Data = pd.read_sql(sql='SELECT * FROM CESM_Data_Thesis', con=engine)
df_CESM_Data.head()

## Close the Connection

In [ ]:
#Once the metadata is loaded by sqlalchemy, you can close the connection
#Close connection
del engine

## Outlier Detection

Having worked extensively with this dataset, we know that there are three outliers that can be dropped right off the bat:

In [ ]:
#Check for outliers in model_cost
df.model_cost.describe()

In [ ]:
plt.boxplot(df["model_cost"])
plt.title('Model Cost Boxplot')
plt.show()

In [ ]:
#Look at top 5 model_cost outliers
df_top5 = df.nlargest(5, 'model_cost')
with pd.option_context("display.max_columns", 1000):
   display (df_top5.model_cost)

## Drop Outliers

We confirmed with software engineering that the top 2 model costs are outliers from the first two test runs. The third highest cost is also an outlier at approximately 150% above the next closest. Since this value  is one out of 6,000+ entires, I will drop it also:

In [ ]:
#Drop the three outliers above 65,000:
df_NoOutlier = df[df .model_cost <= 65000]
len(df_NoOutlier)

In [ ]:
#Confirm
plt.boxplot(df_NoOutlier["model_cost"])
plt.title('Model Cost Boxplot - No Outliers')
plt.show()

## Write df without Outliers to CSV

In [ ]:
#Write as df_NoOutlier to a csv file
#****
#Set new file name
df_NoOutlier.to_csv('df_NoOutlier-Thesis.csv', sep=',')